In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from rubric_assessment_app import app
app()

https://canvas.ubc.ca/api/graphql

Token Valid: Alison Myers (185698)



In [2]:
from initial_requests import get_initial_info
from rubric_assessment_app import GRAPH_URL, KEY
from helpers import _return_single_dict_match
data = get_initial_info(GRAPH_URL, 5433, KEY)


In [3]:
assignments = data['data']['course']['assignmentsConnection']['nodes']

In [5]:
selected_assignment = _return_single_dict_match(assignments, "_id", "188809")

In [6]:
selected_assignment

{'_id': '188809',
 'name': 'A Written Assignment',
 'dueAt': None,
 'expectsSubmission': True,
 'pointsPossible': 9.0,
 'state': 'published',
 'rubric': {'id': 'UnVicmljLTI5OTU1',
  'pointsPossible': 9.0,
  'title': 'Some Rubric',
  '_id': '29955',
  'criteria': [{'_id': '_9408',
    'description': 'Description of criterion',
    'longDescription': '',
    'points': 4.0,
    'ratings': [{'_id': 'blank',
      'longDescription': '',
      'description': 'Superior Work',
      'points': 4.0},
     {'_id': '_1882',
      'longDescription': '',
      'description': 'Meets Expectations',
      'points': 3.0},
     {'_id': '_2024',
      'longDescription': '',
      'description': 'No effort',
      'points': 1.0},
     {'_id': 'blank_2',
      'longDescription': '',
      'description': 'Really bad',
      'points': 0.0}]},
   {'_id': '29955_5920',
    'description': 'Quality of Argument',
    'longDescription': '',
    'points': 5.0,
    'ratings': [{'_id': '29955_479',
      'longDescript

In [7]:
submissions = selected_assignment.get("submissionsConnection").get("nodes")

In [10]:
submissions

[{'_id': '92230271',
  'score': 3.0,
  'state': 'graded',
  'submissionStatus': 'submitted',
  'submittedAt': '2022-05-30T14:03:08-07:00',
  'attempt': 1,
  'grade': '3',
  'user': {'name': 'LTS StudentOne', '_id': '93991', 'sisId': None},
  'commentsConnection': {'nodes': [{'comment': 'Overall good work, but as discussed, you did not submit on time, and will not receive full marks.',
     'author': {'name': 'Peter Lukasik', '_id': '424'}}]},
  'rubricAssessmentsConnection': {'nodes': [{'_id': '2257472',
     'assessmentRatings': [{'_id': 'blank',
       'comments': 'Excellent description',
       'commentsHtml': 'Excellent description',
       'description': 'Superior Work',
       'points': 4.0,
       'criterion': {'_id': '_9408',
        'description': 'Description of criterion',
        'longDescription': '',
        'points': 4.0}},
      {'_id': '29955_5192',
       'comments': 'Solid phrasing',
       'commentsHtml': 'Solid phrasing',
       'description': 'Meets Expectations',

In [37]:
submissions

def get_rubric_assessment(submission):
    user = submission["user"]
    user_name = user["name"]
    user_id = user["_id"]
    user_sis = user["sisId"]
    score = submission["score"]
    attempt = submission["attempt"]
    
    new_dict = {"user_id":  user["_id"],
           "user_name":  user["name"],
           "user_sis_id": user["sisId"],
           "user_score": submission["score"],
           "submission_attempt": submission["attempt"],
            "submission_timestamp": submission["submittedAt"],
            "submission_status": submission["submissionStatus"]}
    
    try:
        rubric_details = submission["rubricAssessmentsConnection"]["nodes"]
        
        for i in rubric_details:
            assessment_ratings = i.get("assessmentRatings")
            
            points_list = []
            descriptive_list = []
            comments_list = []
            
            
            
            for j in assessment_ratings:
                criteria = j["criterion"]["description"]
                
                points = {criteria: j.get("points")}
                descriptive = {criteria: j.get("description")}
                comment = {criteria: j.get("comments")}
                
                points_list.append(points)
                descriptive_list.append(descriptive)
                comments_list.append(comment)
                
            new_dict.update({"points": points_list,
                            "descriptions": descriptive_list,
                            "comments": comments_list})
            
            return(new_dict)
        
    except Exception as err:
        print(f"Error: {err}")



In [38]:
assessment_reviews = []

for j in submissions:
    assessment_reviews.append(get_rubric_assessment(j))
    

In [40]:
import pandas as pd
df = pd.DataFrame(assessment_reviews)

In [41]:
points_df = df.drop(['descriptions', 'comments'], axis=1)

In [43]:
points_df.explode('points')

,user_id,user_name,user_sis_id,user_score,submission_attempt,submission_timestamp,submission_status,points
0,93991,LTS StudentOne,None,3.0,1,2022-05-30T14:03:08-07:00,submitted,{'Description of criterion': 4.0}
0,93991,LTS StudentOne,None,3.0,1,2022-05-30T14:03:08-07:00,submitted,{'Quality of Argument': 3.0}
1,93992,LTS StudentTwo,None,6.0,1,2022-05-30T14:04:13-07:00,submitted,{'Description of criterion': 3.0}
1,93992,LTS StudentTwo,None,6.0,1,2022-05-30T14:04:13-07:00,submitted,{'Quality of Argument': 3.0}
2,93994,LtsStudentThree LtsStudentThree,None,9.0,1,2022-05-30T14:09:02-07:00,submitted,{'Description of criterion': 4.0}
2,93994,LtsStudentThree LtsStudentThree,None,9.0,1,2022-05-30T14:09:02-07:00,submitted,{'Quality of Argument': 5.0}
3,93995,LtsStudentFour LtsStudentFour,None,4.0,1,2022-05-30T14:12:28-07:00,submitted,{'Description of criterion': 1.0}
3,93995,LtsStudentFour LtsStudentFour,None,4.0,1,2022-05-30T14:12:28-07:00,submitted,{'Quality of Argument': 3.0}
4,93997,LtsStudentFive LtsStudentFive,None,8.0,1,2020-07-24T13:36:59-07:00,submitted,{'Description of criterion': 3.0}
4,93997,LtsStudentFive LtsStudentFive,None,8.0,1,2020-07-24T13:36:59-07:00,submitted,{'Quality of Argument': 5.0}
